In [ ]:
import re
import xarray as xr
import numpy as np
import os
import requests
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/friedrich.knuth@rutgers.edu/20180423T142423-RS01SBPS-PC01A-05-ADCPTD102-streamed-adcp_velocity_beam/catalog.html'
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i) for i in x]

In [3]:
# make the output directory
new_dir = 'minute_mean_data/'
if not os.path.isdir(new_dir):
    try:
        os.makedirs(new_dir)
    except OSError:
        if os.path.exists(new_dir):
            pass
        else:
            raise

In [ ]:
# datasets = ['https://opendap.oceanobservatories.org/thredds/dodsC/ooi/friedrich.knuth@rutgers.edu/20180423T142423-RS01SBPS-PC01A-05-ADCPTD102-streamed-adcp_velocity_beam/deployment0001_RS01SBPS-PC01A-05-ADCPTD102-streamed-adcp_velocity_beam_20150529T000000.660000-20150531T235959.850000.nc']
ds = xr.open_mfdataset(datasets)
ds = ds.swap_dims({'obs': 'time'})
ds = ds.chunk({'time': 100})

In [ ]:
time = ds['time'].data
bin_depths = ds['bin_depths'].data.T
east = ds['eastward_seawater_velocity'].data.T
north = ds['northward_seawater_velocity'].data.T
up = ds['upward_seawater_velocity'].data.T

In [ ]:
east

In [ ]:
u_v_w = np.concatenate(east, north, up)
lim = float("%2.2f" % np.nanpercentile(abs(u_v_w), 90))

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 10)
p = ax.pcolormesh(time, bins, east, cmap='RdBu',vmin=-lim,vmax=lim)
ax.set_ylim(20,200)
ax.invert_yaxis()
fig.colorbar(p)

In [ ]:
start = datetime.strptime('2015-05-29','%Y-%m-%d')
end = datetime.strptime('2015-05-30','%Y-%m-%d')

In [ ]:
ds_slice = ds.sel(time=slice(start, end))

In [ ]:
# horizontal_data = np.concatenate(north)


In [ ]:
fig, ax = plt.subplots()
p = ax.pcolormesh(time, bins, north, cmap='RdBu',vmin=-lim,vmax=lim)
ax.set_ylim(20,200)
ax.invert_yaxis()
cb = fig.colorbar(p)
cb.update_ticks()
cb.formatter.set_useOffset(False)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16, 6);
ax.invert_yaxis()
ax.grid()
ax.set_xlim(x[0],x[-1])
sc = plt.scatter(x, y, c=z)
cb = fig.colorbar(sc, ax=ax)
cb.update_ticks()
cb.formatter.set_useOffset(False)
plt.show()

In [ ]:
df = ds.to_dataframe()

In [ ]:
df_east = ds['eastward_seawater_velocity'].to_pandas()

In [ ]:
df_east

In [ ]:
df_east.index.get_level_values

In [ ]:
test = df['eastward_seawater_velocity'].resample('T').mean()

In [ ]:
    pressure_min = pd.DataFrame()
    pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()

In [ ]:
ds.time.min()

In [ ]:
ds.time.max()

In [ ]:
ds1

In [ ]:
time = ds1['time'].values
bins = ds1['bin_depths'].values
north = ds1['northward_seawater_velocity'].values
east = ds1['eastward_seawater_velocity'].values
up = ds1['upward_seawater_velocity'].values

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(time, bins, north)

fig.set_size_inches(16, 6);
# ax.invert_yaxis()
# ax.grid()
# ax.set_xlim(x[0],x[-1])
# sc = plt.scatter(x, y, c=z)
# cb = fig.colorbar(sc, ax=ax)
# cb.update_ticks()
# cb.formatter.set_useOffset(False)
# plt.show()